In [25]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done


In [26]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [27]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [28]:
# Create a temporary view of the DataFrame.
home_sales_data.createOrReplaceTempView('home_sales')

# Check out the temp view

spark.sql("SELECT * from home_sales limit 10").show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [29]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

spark.sql("""SELECT date_built,
              ROUND(AVG(price),2) AS Average_Price
              FROM home_sales
              WHERE bedrooms = 4
              GROUP BY date_built
              ORDER BY date_built""").show()

+----------+-------------+
|date_built|Average_Price|
+----------+-------------+
|      2010|    296800.75|
|      2011|     302141.9|
|      2012|    298233.42|
|      2013|    299999.39|
|      2014|    299073.89|
|      2015|    307908.86|
|      2016|    296050.24|
|      2017|    296576.69|
+----------+-------------+



In [30]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

spark.sql("""SELECT date_built AS Build_Year,
              ROUND(AVG(price),2) AS Average_Price
              FROM home_sales
              WHERE bedrooms = 3
              AND bathrooms = 3
              GROUP BY date_built
              ORDER BY date_built""").show()

+----------+-------------+
|Build_Year|Average_Price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [31]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""SELECT date_built AS Build_Year,
              ROUND(AVG(price),2) AS Average_Price
              FROM home_sales
              WHERE bedrooms = 3
              AND bathrooms = 3
              AND floors = 2
              AND sqft_living >= 2000
              GROUP BY date_built
              ORDER BY date_built""").show()


+----------+-------------+
|Build_Year|Average_Price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [32]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""SELECT view,
            ROUND(AVG(price),2)
            FROM home_sales
            WHERE price >= 350000
            GROUP BY view
            ORDER BY view""").show()

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   0|           403848.51|
|   1|           401044.25|
|   2|           397389.25|
|   3|            398867.6|
|   4|           399631.89|
|   5|           401471.82|
|   6|           395655.38|
|   7|           403005.77|
|   8|           398592.71|
|   9|           401393.34|
|  10|           401868.43|
|  11|           399548.12|
|  12|           401501.32|
|  13|           398917.98|
|  14|           398570.03|
|  15|            404673.3|
|  16|           399586.53|
|  17|           398474.49|
|  18|           399332.91|
|  19|           398953.17|
+----+--------------------+
only showing top 20 rows



In [33]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [34]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [35]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""SELECT view,
            ROUND(AVG(price),2)
            FROM home_sales
            WHERE price >= 350000
            GROUP BY view
            ORDER BY view DESC""").show()



print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
| 100|           1026669.5|
|  99|          1061201.42|
|  98|          1053739.33|
|  97|          1129040.15|
|  96|          1017815.92|
|  95|           1054325.6|
|  94|           1033536.2|
|  93|          1026006.06|
|  92|           970402.55|
|  91|          1137372.73|
|  90|          1062654.16|
|  89|          1107839.15|
|  88|          1031719.35|
|  87|           1072285.2|
|  86|          1070444.25|
|  85|          1056336.74|
|  84|          1117233.13|
|  83|          1033965.93|
|  82|           1063498.0|
|  81|          1053472.79|
+----+--------------------+
only showing top 20 rows

--- 0.5572340488433838 seconds ---


In [36]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_data.write.partitionBy("date_built").mode("overwrite").parquet("parquet_home_sales")

In [37]:
# 11. Read the parquet formatted data.
home_sales_parquet_df=spark.read.parquet('parquet_home_sales')

In [38]:
# 12. Create a temporary table for the parquet data.
home_sales_parquet_df.createOrReplaceTempView('parquet_home_sales_table')

In [39]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""SELECT view,
            ROUND(AVG(price),2)
            FROM parquet_home_sales_table
            WHERE price >= 350000
            GROUP BY view
            ORDER BY view DESC""").show()



print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
| 100|           1026669.5|
|  99|          1061201.42|
|  98|          1053739.33|
|  97|          1129040.15|
|  96|          1017815.92|
|  95|           1054325.6|
|  94|           1033536.2|
|  93|          1026006.06|
|  92|           970402.55|
|  91|          1137372.73|
|  90|          1062654.16|
|  89|          1107839.15|
|  88|          1031719.35|
|  87|           1072285.2|
|  86|          1070444.25|
|  85|          1056336.74|
|  84|          1117233.13|
|  83|          1033965.93|
|  82|           1063498.0|
|  81|          1053472.79|
+----+--------------------+
only showing top 20 rows

--- 1.1013338565826416 seconds ---


In [40]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

DataFrame[]

In [41]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False